# Pipeline ML Modulare per Cybersecurity: Guida Completa

Questo notebook è una guida interattiva a una pipeline di machine learning end-to-end per la classificazione di traffico di rete. È stato progettato con tre principi chiave in mente:

- **Modularità**: Ogni fase logica (preprocessing, training, predizione) è isolata nel proprio modulo Python (`.py`), seguendo il Single Responsibility Principle. Questo rende il codice più pulito, facile da mantenere e da testare.
- **Configurabilità**: Tutta la pipeline è controllata da un singolo file, `config.py`. Questo agisce come un "pannello di controllo" per cambiare dataset, selezionare feature e fare tuning degli iperparametri senza toccare il codice della logica.
- **Estensibilità**: La struttura è pensata per essere facilmente adattabile a nuovi dataset o a nuove architetture di modelli.

## 1. Architettura e Scelte Tecniche

La pipeline è composta dai seguenti elementi:

- `create_synthetic_data.py`: Uno script per generare dati di esempio realistici. Utile per testare la pipeline senza bisogno di un dataset reale.
- `config.py`: Il cuore della configurazione. Qui si definiscono i percorsi, le colonne da usare e gli iperparametri per il training.
- `preprocessing/process.py`: Contiene tutta la logica per caricare i dati e trasformarli in un formato numerico che il modello può comprendere (es. anonimizzazione IP, one-hot encoding).
- `training/train.py`: Gestisce la creazione del modello, la grid search per trovare i migliori iperparametri e il salvataggio del modello più performante.
- `prediction/predict.py`: Carica un modello salvato e lo usa per fare predizioni su nuovi dati, applicando la stessa logica di preprocessing.

**Librerie Utilizzate:**
- **Pandas**: Per la manipolazione efficiente dei dati tabulari.
- **Scikit-learn**: Per utility di preprocessing standard come `LabelEncoder` e per la divisione dei dati in set di training e test.
- **TensorFlow (con Keras)**: Come framework di deep learning. È stato scelto per la sua flessibilità e facilità d'uso nel costruire modelli di rete neurale.
- **Faker**: Per generare dati fittizi ma realistici (come gli indirizzi IP).

## 2. Setup Iniziale

Installiamo le librerie necessarie per eseguire l'intera pipeline.

In [ ]:
# Installa le dipendenze
!pip install pandas scikit-learn tensorflow faker

## 3. Esecuzione della Pipeline

Ora eseguiamo la pipeline passo dopo passo, importando le funzioni dai nostri moduli.

### Fase 1: Generazione del Dataset Sintetico

Creiamo un dataset di esempio per poter lavorare. Lo script salverà il file in `data/cybersecurity_data.csv`.

In [ ]:
from create_synthetic_data import generate_synthetic_data
generate_synthetic_data()

### Fase 2: Preprocessing dei Dati

Questa è una fase cruciale. Il modulo `process.py` esegue diverse trasformazioni chiave:

- **Anonimizzazione IP**: I modelli di machine learning non possono processare stringhe come '192.168.1.1'. Questa funzione mappa ogni indirizzo IP univoco a un ID numerico (es. 1, 2, 3...). La mappa (`ip -> id` e `id -> ip`) viene salvata per poterla riutilizzare durante la predizione.
- **One-Hot Encoding**: La colonna `protocollo` contiene valori come 'TCP', 'UDP'. Per evitare che il modello interpreti una relazione numerica errata (es. TCP=0, UDP=1, ICMP=2), trasformiamo questa colonna in più colonne binarie (`protocollo_TCP`, `protocollo_UDP`), dove solo una può essere 1 (vera).
- **Encoding del Target**: Similmente, le etichette di attacco ('normal', 'dos', ecc.) vengono mappate in interi, necessari per la funzione di loss del modello.

In [ ]:
from preprocessing.process import preprocess_data

X, y, _, _ = preprocess_data()

if X is not None:
    print("\n--- Anteprima Feature (X) Processate ---")
    display(X.head())

### Fase 3: Training e Tuning degli Iperparametri

Ora addestriamo il modello. Il modulo `train.py` esegue una **Grid Search**: testa diverse combinazioni di iperparametri definite in `config.py` per trovare la configurazione migliore.

In [ ]:
from training.train import train_and_evaluate

_, _ = train_and_evaluate()

### Guida alla Configurazione degli Iperparametri (`config.py`)

La grid search testa le combinazioni dei valori che inserisci nelle liste in `TRAINING_CONFIG['hyperparameters']`. Ecco una guida su come sceglierli:

- `activation` (`'relu'`, `'tanh'`): Funzione di attivazione dei neuroni. `'relu'` è un'ottima scelta di default, veloce ed efficiente. `'tanh'` può essere utile in reti più profonde ma è più costosa computazionalmente.

- `batch_size` (`16`, `32`): Numero di campioni processati prima di aggiornare il modello. 
  - **Valori Bassi (es. 16, 32)**: L'addestramento è più lento ma può convergere meglio (generalizzare) perché gli aggiornamenti sono più frequenti e "rumorosi". Ideale per dataset complessi e non enormi.
  - **Valori Alti (es. 128, 256)**: L'addestramento è molto più veloce. Utile per dataset molto grandi e stabili, ma c'è il rischio di convergere a soluzioni non ottimali.

- `epochs` (`50`): Numero di volte in cui il modello vedrà l'intero dataset. Un numero troppo basso porta a *underfitting* (il modello non impara abbastanza), un numero troppo alto a *overfitting* (il modello impara a memoria i dati di training e non generalizza). Il valore giusto si trova sperimentalmente, monitorando la performance sul set di validazione.

- `learning_rate` (`0.001`, `0.01`): Controlla la dimensione dei "passi" durante l'ottimizzazione. 
  - **Valore Basso (es. 0.0001)**: Passi piccoli, training lento ma più probabilità di trovare un buon minimo. 
  - **Valore Alto (es. 0.01)**: Passi grandi, training veloce ma si rischia di "saltare" la soluzione ottimale.

- `hidden_layer_size` (`32`, `64`): Numero di neuroni nel layer nascosto. Controlla la **capacità** del modello. Più neuroni possono apprendere pattern più complessi, ma aumentano il rischio di overfitting e il costo computazionale.

### Fase 4: Predizione su un Nuovo Campione

Infine, usiamo il modello migliore salvato per fare una predizione su un nuovo dato, che non ha mai visto prima.

In [ ]:
from prediction.predict import predict

# Creiamo un campione di dati fittizio
sample = {
    "ip_sorgente": "192.168.1.10",
    "ip_destinazione": "10.0.0.5",
    "porta_sorgente": 12345,
    "porta_destinazione": 443,
    "protocollo": "UDP",
    "byte_inviati": 250,
    "byte_ricevuti": 1800,
    "pacchetti_inviati": 8,
    "pacchetti_ricevuti": 12,
}

prediction_label = predict(sample)

if prediction_label:
    print(f"\nRISULTATO FINALE: Il campione è stato classificato come: '{prediction_label}'")

## 4. Come Adattare la Pipeline al Tuo Dataset

La forza di questa architettura è la facilità con cui puoi usarla per i tuoi dati. Ecco come fare.

### Passaggio 1: Aggiorna `config.py`

Immagina di avere un nuovo dataset `my_traffic.csv` con le seguenti colonne: `['Timestamp', 'SourceIP', 'DestIP', 'SourcePort', 'DestPort', 'ProtocolType', 'PacketCount', 'TrafficType']`.

Per adattare la pipeline, dovrai modificare `DATA_CONFIG` in `config.py` così:

```python
DATA_CONFIG = {
    # 1. Cambia il percorso del file
    "dataset_path": "data/my_traffic.csv",

    # 2. Definisci le tue feature numeriche
    "numeric_feature_columns": [
        "SourcePort",
        "DestPort",
        "PacketCount"
    ],
    
    # 3. Definisci le tue feature categoriche
    "one_hot_encode_columns": ["ProtocolType"],

    # 4. Definisci le tue colonne IP
    "ip_columns_to_anonymize": ["SourceIP", "DestIP"],

    # 5. Definisci la tua colonna target
    "target_column": "TrafficType",

    "anonymize_target": True,
}
```
**Fatto!** Eseguendo di nuovo il notebook o gli script, la pipeline userà automaticamente il tuo nuovo dataset e le tue colonne.

### Passaggio 2: Creare Nuove Feature (Feature Engineering)

Spesso, le feature migliori non sono quelle originali, ma quelle che creiamo noi. Immaginiamo di voler creare una feature `is_well_known_port` che è `1` se la porta di destinazione è una porta comune (es. 80, 443) e `0` altrimenti.

Puoi farlo modificando leggermente `preprocessing/process.py`:

1. Apri `preprocessing/process.py`.
2. Trova la sezione dove viene caricato il DataFrame `df`.
3. **Aggiungi il tuo codice** per creare la nuova colonna, prima che le feature vengano assemblate in `X`.

```python
# Esempio da aggiungere in process.py dopo il caricamento di df

well_known_ports = [80, 443, 22, 21, 53]
df['is_well_known_port'] = df['porta_destinazione'].isin(well_known_ports).astype(int)
```

4. Infine, **aggiorna `config.py`** per includere la tua nuova feature nella lista delle colonne numeriche:

```python
"numeric_feature_columns": [
    "porta_sorgente",
    "porta_destinazione",
    # ... altre colonne ...
    "is_well_known_port" # <-- Aggiungi qui la tua nuova feature
],
```
Questo approccio ti dà la flessibilità di creare logiche di preprocessing complesse mantenendo la configurazione semplice e centralizzata.